In [1]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')

# Read and Decode BBCI Data

This tutorial shows how to read and decode BBCI data.

## Setup logging to see outputs

In [2]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
log = logging.getLogger()


## Load and preprocess data

In [3]:
import mne
from braindecode.datasets.bbci import BBCIDataset
import numpy as np
from braindecode.mne_ext.signalproc import resample_cnt, mne_apply
from braindecode.datautil.signalproc import bandpass_cnt, exponential_running_standardize

from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne
from collections import OrderedDict
# can also give lists of marker codes in case a class has multiple marker codes...
name_to_code = OrderedDict([('Right', 1), ('Left', 2), ('Rest', 3), ('Feet', 4)])

train_filename = '/home/schirrmr/data/BBCI-without-last-runs/BhNoMoSc1S001R01_ds10_1-12.BBCI.mat'
cnt = BBCIDataset(train_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()
# Remove stimulus channel
cnt = cnt.pick_channels(['C3', 'CPz', 'C4'])
cnt = resample_cnt(cnt, 250)
# mne apply will apply the function to the data (a 2d-numpy-array)
# have to transpose data back and forth, since
# exponential_running_standardize expects time x chans order
# while mne object has chans x time order
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)
train_set = create_signal_target_from_raw_mne(cnt, name_to_code, [-500,4000])

test_filename = '/home/schirrmr/data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat'
cnt = BBCIDataset(test_filename, load_sensor_names=['C3', 'CPz', 'C4']).load()
# Remove stimulus channel
cnt = cnt.pick_channels(['C3', 'CPz', 'C4'])
cnt = resample_cnt(cnt, 250)
cnt = mne_apply(lambda a: exponential_running_standardize(
    a.T, init_block_size=1000,factor_new=0.001, eps=1e-4).T,
    cnt)
test_set = create_signal_target_from_raw_mne(cnt, name_to_code, [-500,4000])

Creating RawArray with float64 data, n_channels=3, n_times=3451320
    Range : 0 ... 3451319 =      0.000 ...  6902.638 secs
Ready.
2017-07-04 12:39:06,201 WARNING : This is not causal, uses future data....
2017-07-04 12:39:06,204 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=1725660
    Range : 0 ... 1725659 =      0.000 ...  6902.636 secs
Ready.
2017-07-04 12:39:10,347 INFO : Trial per class:
Counter({'Feet': 225, 'Rest': 224, 'Left': 224, 'Right': 224})
Creating RawArray with float64 data, n_channels=3, n_times=617090
    Range : 0 ... 617089 =      0.000 ...  1234.178 secs
Ready.
2017-07-04 12:39:10,991 WARNING : This is not causal, uses future data....
2017-07-04 12:39:10,999 INFO : Resampling from 500.000000 to 250.000000 Hz.
Creating RawArray with float64 data, n_channels=3, n_times=308545
    Range : 0 ... 308544 =      0.000 ...  1234.176 secs
Ready.
2017-07-04 12:39:11,947 INFO : Trial per class:
Counter({'Rest'

<div class="alert alert-info">

In case of start and stop markers, provide a `name_to_stop_codes` dictionary (same as for the start codes in this example) as a final argument to `create_signal_target_from_raw_mne`. Also see [Read and Decode BBCI Data with Start-Stop-Markers Tutorial](BBCI_Data_Start_Stop.html)


</div>

In [4]:
from braindecode.datautil.splitters import split_into_two_sets

train_set, valid_set = split_into_two_sets(train_set, first_set_fraction=0.8)


## Create the model

In [5]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds, to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 800
in_chans = 3
n_classes = 4
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length,
                        final_conv_length=30).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()

## Setup optimizer and iterator

In [6]:
from torch import optim

optimizer = optim.Adam(model.parameters())

In [7]:
from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(np.ones((2, 3, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))

267 predictions per input/trial


In [8]:
from braindecode.datautil.iterators import CropsFromTrialsIterator
iterator = CropsFromTrialsIterator(batch_size=32,input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

## Setup Monitors, Loss function, Stop Criteria

In [9]:
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression


loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2)[:,:,0], targets)

model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length), RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
exp = Experiment(model, train_set, valid_set, test_set, iterator, loss_function, optimizer, model_constraint,
          monitors, stop_criterion, remember_best_column='valid_misclass',
          run_after_early_stop=True, batch_modifier=None, cuda=cuda)

## Run experiment

In [10]:
exp.run()

2017-07-04 12:39:19,972 INFO : Run until first stop...
2017-07-04 12:39:21,295 INFO : Epoch 0
2017-07-04 12:39:21,297 INFO : train_loss                9.68400
2017-07-04 12:39:21,298 INFO : valid_loss                9.49615
2017-07-04 12:39:21,300 INFO : test_loss                 9.54656
2017-07-04 12:39:21,301 INFO : train_sample_misclass     0.75040
2017-07-04 12:39:21,303 INFO : valid_sample_misclass     0.74911
2017-07-04 12:39:21,304 INFO : test_sample_misclass      0.75250
2017-07-04 12:39:21,306 INFO : train_misclass            0.75070
2017-07-04 12:39:21,307 INFO : valid_misclass            0.74860
2017-07-04 12:39:21,309 INFO : test_misclass             0.75000
2017-07-04 12:39:21,310 INFO : runtime                   0.00000
2017-07-04 12:39:21,312 INFO : 
2017-07-04 12:39:21,314 INFO : New best valid_misclass: 0.748603
2017-07-04 12:39:21,316 INFO : 
2017-07-04 12:39:23,978 INFO : Epoch 1
2017-07-04 12:39:23,984 INFO : train_loss                1.03746
2017-07-04 12:39:23,986

We arrive at 75% accuracy.